In [3]:
import os
os.path.dirname('00_test.ipynb')

''

In [5]:
import boto3

client = boto3.client('s3')
client

client.list_objects_v2(Bucket='peptidescreening', Prefix='safety_check/ex')

NoCredentialsError: Unable to locate credentials

In [1]:
import pandas as pd

In [ ]:
import pandas as pd

in_file = '/home/platform/project/mrnadesign_platform/mrnadesign_api/workspace/analysis_script/user/1732192698_983D/sequences/input_seq.tsv'
pd.read_csv(in_file, sep='\t', index_col=None)

in_file = '/home/platform/project/mrnadesign_platform/mrnadesign_api/workspace/analysis_script/demo_user/demouser_prediction_task0003/sequences/input_seq.tsv'
pd.read_csv(in_file, sep='\t', index_col=None)

,seq_id,5utr,cds,3utr,seq_acc
0,0,CUAAUGCCAUGAUCCAGGUGACAUGUAGAAGCUUGGAUCAGAUGCU...,AUGCCCAUGCCCAUCGGCAGCAAGGAGAGGCCCACCUUCUUCGAGA...,AUUGAUUUU,SEQ000000
1,1,CUAAUGCCAUGAUCCAACAUGUGGAAGCUUGGAUCAGAUGCUGCAC...,AUGCCCAUGCCCAUCGGCAGCAAGGAGAGGCCCACCUUCUUCGAGA...,AUUGAUUGGGGUAAUAAAGGGU,SEQ000001


In [ ]:
import pandas as pd

# Sample updated data with new start and end values
data = {
    'Category': ['A', 'A', 'A', 'A', 'A'],
    'last': ['aa1', 'bb1', 'aa2', 'bb2', 'cc'],
    'Description': ['first', 'second', 'third', 'fourth', 'fifth'],
    'start': [4, 9, 11, 13, 15],  # 1-based index
    'end': [9, 17, 20, 19, 19]     # 1-based index
}

# Given sequence
sequence = "AAAAABBBBBCCCCCDDDDDEEEEE"

# Create a DataFrame
df = pd.DataFrame(data)

# Identify the grouping key
grouping_key = 'Category'

# Group by the identified key and convert to the desired format
result = {
    category: sorted(
        [
            {**{col: row[col] for col in df.columns if col != grouping_key},
             'substring': sequence[row['start'] - 1: row['end']]}  # Extract substring
            for index, row in group.iterrows()
        ],
        key=lambda x: (x['start'], x['end'])  
    )
    for category, group in df.groupby(grouping_key)
}

# Initialize a list for the node information
ddf_list = {}
for type in list(result.keys()):
    ddf = pd.DataFrame()

    color_info = [0] * len(sequence)
    color_count = 1
    belongings_info = [set() for _ in range(len(sequence))]
    highlight_info = [[_, _] for _ in range(len(sequence))]

    type_result = result.get(type, [])
    for entry_idx, entry in enumerate(type_result):  # Processing each entry
        start = entry['start']  # start is from 1, not 0
        end = entry['end']
        color_info[start] = color_count
        for _i in range(start, end + 1):
            i = _i - 1
            belongings_info[i].add(entry_idx)
            highlight_info[i][0] = min(highlight_info[i][0], start)
            highlight_info[i][1] = max(highlight_info[i][1], end)
        color_count += 1

    ddf['node'] = [i for i in sequence]
    ddf['color'] = color_info
    ddf['belongings'] = [tuple(sorted(b)) for b in belongings_info]  # Convert set to tuple for hashing
    ddf['highlight_range'] = [tuple(sorted(b)) for b in highlight_info]

    # # Create a new column to track if a row can be grouped
    # ddf['can_group'] = (ddf['color'].shift() == ddf['color']) & (ddf['belongings'].shift() == ddf['belongings']) & (ddf['highlight_range'].shift() == ddf['highlight_range'])
    # ddf['group_id'] = (~ddf['can_group']).cumsum()

    # # Group by the unique group id and aggregate
    # ddf_grouped = ddf.groupby(['group_id']).agg({
    #     'color': 'first',  # Keep the first color
    #     'belongings': 'first',  # Keep the first belongings
    #     'highlight_range': 'first',  # Keep the first highlight_range
    #     'node': ''.join  # Concatenate node values
    # }).reset_index(drop=True)

    # ddf_grouped['belongings'] = [list(i) for i in ddf_grouped['belongings']]
    # ddf_grouped['highlight_range'] = [list(i) for i in ddf_grouped['highlight_range']]

    ddf_list[type] = ddf_grouped
    break  # Remove this break if you want to process all categories

ddf_grouped

,color,belongings,highlight_range,node
0,0,[],"[0, 0]",A
1,0,[],"[1, 1]",A
2,0,[],"[2, 2]",A
3,0,[0],"[3, 9]",A
4,1,[0],"[4, 9]",A
5,0,[0],"[4, 9]",BBB
6,0,"[0, 1]","[4, 17]",B
7,2,[1],"[9, 17]",B
8,0,"[1, 2]","[9, 20]",C
9,3,"[1, 2]","[9, 20]",C


In [ ]:
import argparse
import configparser

# parser = argparse.ArgumentParser()
# parser.add_argument('--config', type=str, default='/home/platform/project/mrnadesign_platform/mrnadesign_api/workspace/analysis_script/user/1730469766_QLH7/task_prediction_config.ini')

# args = parser.parse_args()
# user_config = args.config

config = configparser.ConfigParser()
config.read('/home/platform/project/mrnadesign_platform/mrnadesign_api/workspace/analysis_script/user/1730469766_QLH7/task_prediction_config.ini')
config.get('ires', 'target_species')


'Homo sapiens'

In [ ]:
import pandas as pd
import numpy as np

path = '/home/platform/project/mrnadesign_platform/mrnadesign_api/workspace/mrnadesign_database/'

In [ ]:
df1 = pd.read_csv(path + 'Antigen_se.csv')
print(df1.shape)
df2 = pd.read_csv(path + 'TAntigendb_se.csv')
print(df2.shape)
df3 = pd.read_csv(path + 'Tantigen_rna_seq.csv')
print(df3.shape)
df4 = pd.read_csv(path + 'Complete_3_UTR_ARE_Data/Complete_3_UTR_Data.csv')
print(df4.shape)

(505, 20)
(4295, 13)
(4306, 5)
(4884, 10)


In [ ]:
df1['Sequence'].to_numpy()

array(['\nMTRNKFIPNKFSIISFSVLLFAISSSQAIEVNAMNEHYTESDIKRNHKTEKNKTEKEKFK\nDSINNLVKTEFTNETLDKIQQTQDLLKKIPKDVLEIYSELGGEIYFTDIDLVEHKELQDL\nSEEEKNSMNSRGEKVPFASRFVFEKKRETPKLIINIKDYAINSEQSKEVYYEIGKGISLD\nIISKDKSLDPEFLNLIKSLSDDSDSSDLLFSQKFKEKLELNNKSIDINFIKENLTEFQHA\nFSLAFSYYFAPDHRTVLELYAPDMFEYMNKLEKGGFEKISESLKKEGVEKDRIDVLKGEK\nALKASGLVPEHADAFKKIARELNTYILFRPVNKLATNLIKSGVATKGLNVHGKSSDWGPV\nAGYIPFDQDLSKKHGQQLAVEKGNLENKKSITEHEGEIGKIPLKLDHLRIEELKENGIIL\nKGKKEIDNGKKYYLLESNNQVYEFRISDENNEVQYKTKEGKITVLGEKFNWRNIEVMAKN\nVEGVLKPLTADYDLFALAPSLTEIKKQIPQKEWDKVVNTPNSLEKQKGVTNLLIKYGIER\nKPDSTKGTLSNWQKQMLDRLNEAVKYTGYTGGDVVNHGTEQDNEEFPEKDNEIFIINPEG\nEFILTKNWEMTGRFIEKNITGKDYLYYFNRSYNKIAPGNKAYIEWTDPITKAKINTIPTS\nAEFIKNLSSIRRSSNVGVYKDSGDKDEFAKKESVKKIAGYLSDYYNSANHIFSQEKKRKI\nSIFRGIQAYNEIENVLKSKQIAPEYKNYFQYLKERITNQVQLLLTHQKSNIEFKLLYKQL\nNFTENETDNFEVFQKIIDEK\n',
       '\nMKKRKVLIPLMALSTILVSSTGNLEVIQAEVKQENRLLNESESSSQGLLGYYFSDLNFQA\nPMVVTSSTTGDLSIPSSELENIPSENQYFQSAIWSGFIKVKKSDEYTFATSADNHVTMWV\nDDQEVINKASNSNKIRLEKGRLYQI

In [ ]:
np.unique([df1['Antigen Name'][i] + df1['Antigen Sequence'][i] for i in range(505)]).shape

(485,)

In [ ]:
df1['Antigen Name'], df1['Antigen Sequence']

TypeError: 'map' object is not subscriptable

In [ ]:
merged_df = pd.merge(df2, df3, how='outer', on=['AgACC'])

merged_df['Antigen Name_x'] = merged_df['Antigen Name_x'].combine_first(merged_df['Antigen Name_y'])
merged_df = merged_df.drop(columns=['Antigen Name_y'])
merged_df.rename(columns={'Antigen Name_x': 'Antigen Name'}, inplace=True)
# merged_df = merged_df.to_numpy()

In [ ]:
try:
    float('asd')
except ValueError:
    print('sdf')

sdf


In [ ]:
merged_df.shape

(4307, 16)

In [ ]:
ddf1 = pd.DataFrame([
    ['a', 1, 11],
    ['b', 2, 22],
    ['c', 3, 33]
], columns=['letter', 'n1', 'n2'])
ddf1

,letter,n1,n2
0,a,1,11
1,b,2,22
2,c,3,33


In [ ]:
ddf2 = pd.DataFrame([
    ['a', 111, 1111],
    ['c', 333, 3333],
    ['d', 444, 4444]
], columns=['letter', 'n1', 'n4'])
ddf2

,letter,n1,n4
0,a,111,1111
1,c,333,3333
2,d,444,4444


In [ ]:
merged_df = pd.merge(ddf1, ddf2, how='outer', on=['letter'])
merged_df 

,letter,n1_x,n2,n1_y,n4
0,a,1.0,11.0,111.0,1111.0
1,b,2.0,22.0,NaN,NaN
2,c,3.0,33.0,333.0,3333.0
3,d,NaN,NaN,444.0,4444.0


In [ ]:
merged_df['n1_x'] = merged_df['n1_x'].combine_first(merged_df['n1_y'])
merged_df = merged_df.drop(columns=['n1_y'])
merged_df.rename(columns={'n1_x': 'n1'}, inplace=True)
merged_df

,letter,n1,n2,n4
0,a,1.0,11.0,1111.0
1,b,2.0,22.0,NaN
2,c,3.0,33.0,3333.0
3,d,444.0,NaN,4444.0
